In [2]:
!pip install -r requirements.txt

E1129 17:09:12.501108927     145 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1638205752.501041319","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1638205752.501036059","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
  Using cached ipyleaflet-0.14.0-py2.py3-none-any.whl (3.3 MB)
  Using cached pygeos-0.11.1-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.1 MB)
  Using cached traittypes-0.2.1-py2.py3-none-any.whl (8.6 kB)


In [3]:
# import os
# os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

import pandas as pd
import numpy as np
import geopandas as gpd
# import fiona
import datetime as dt

from utils import *

import calitp
from calitp.tables import tbl
from siuba import *

### Metric

The % of [wheelchair-using] Californians that are within 1/4 mi of a transit stop that is indicated to be explicitly accessible in a static GTFS feed and served by at least one explicitly accessible trip.

In [35]:
accessible_stops_trips = get_accessible_stops_and_trips()

### Census Data

In [15]:
variable_range = ['0' + str(x) if x < 10 else str(x) for x in range(1,34)]

In [37]:
## https://api.census.gov/data/2019/acs/acs5/variables.html
ambulatory_disability_vars = [f'B18105_0{x}E' for x in variable_range]

In [38]:
ambulatory_disability_vars = ','.join(ambulatory_disability_vars)

In [39]:
ambulatory_disability_df = get_census_ca_counties(ambulatory_disability_vars)

In [40]:
ambulatory_disability_df.head(3)

,B18105_001E,B18105_002E,B18105_003E,B18105_004E,B18105_005E,B18105_006E,B18105_007E,B18105_008E,B18105_009E,B18105_010E,...,B18105_027E,B18105_028E,B18105_029E,B18105_030E,B18105_031E,B18105_032E,B18105_033E,state,county,tract
0,1901,1063,208,0,208,310,8,302,366,26,...,354,75,29,46,48,15,33,6,47,503
1,6111,3287,871,30,841,996,0,996,1079,94,...,875,202,41,161,144,68,76,6,47,504
2,6099,3242,887,36,851,833,0,833,1090,83,...,979,198,29,169,252,72,180,6,47,505


In [41]:
ambulatory_disability_df['total_pop'] = ambulatory_disability_df['B18105_001E']
disability_cols = ['B18105_004E', 'B18105_007E', 'B18105_010E', 
                  'B18105_013E', 'B18105_016E', 'B18105_020E',
                   'B18105_023E', 'B18105_026E', 'B18105_029E',
                   'B18105_032E'
                  ]
ambulatory_disability_df['total_disabled'] = ambulatory_disability_df[disability_cols].sum(axis=1)
ambulatory_disability_summary = ambulatory_disability_df[['state', 'county', 'tract', 'total_pop', 'total_disabled']]

In [42]:
ambulatory_disability_summary

,state,county,tract,total_pop,total_disabled
0,6,47,503,1901,180
1,6,47,504,6111,423
2,6,47,505,6099,387
3,6,47,701,2783,129
4,6,47,1002,14787,1384
...,...,...,...,...,...
3,6,15,201,3279,377
4,6,15,203,1088,186
5,6,15,104,2220,211
6,6,15,105,6093,641


In [43]:
# ambulatory_disability_summary.to_parquet('tract_disability.parquet')

In [44]:
# ambulatory_disability_summary = pd.read_parquet('tract_disability.parquet')

## Block Group Pop

In [45]:
total_pop_var = 'B01001_001E'

In [54]:
blockgrp_pop = get_census_ca_counties(total_pop_var, 'block%20group')

In [55]:
blockgrp_pop = blockgrp_pop.rename(columns={'B01001_001E': 'block_grp_pop'})

In [56]:
# blockgrp_pop.to_parquet('pop_block_grp.parquet')

In [57]:
# blockgrp_pop = pd.read_parquet('./pop_block_grp.parquet')

In [58]:
blockgrp_pop.head(5)

,block_grp_pop,state,county,tract,block group
0,1388,6,47,1200,2
1,1531,6,47,1200,4
2,1405,6,47,1200,1
3,2410,6,47,1401,2
4,1148,6,47,1502,1


geoid example '482012231001'

'48 = state, 201 = county, 2231 = tract, 001 = block group?"

In [61]:
census_joined = blockgrp_pop >> inner_join(_, ambulatory_disability_summary, on=['county', 'tract'])

In [62]:
census_joined.head(5)

,block_grp_pop,state_x,county,tract,block group,state_y,total_pop,total_disabled
0,1388,6,47,1200,2,6,5012,428
1,1531,6,47,1200,4,6,5012,428
2,1405,6,47,1200,1,6,5012,428
3,1312,6,47,1200,3,6,5012,428
4,2410,6,47,1401,2,6,3727,295


In [67]:
census_joined['pct_disabled'] = census_joined['total_disabled'] / census_joined['total_pop']
census_joined['block_grp_disabled_est'] = (census_joined['block_grp_pop'] * census_joined['pct_disabled']).round(0)
census_joined = census_joined.dropna()
census_joined['block_grp_disabled_est'] = census_joined['block_grp_disabled_est'].astype('int64')

/tmp/ipykernel_113/1940049797.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_joined['block_grp_disabled_est'] = census_joined['block_grp_disabled_est'].astype('int64')


In [68]:
census_joined

,block_grp_pop,state_x,county,tract,block group,state_y,total_pop,total_disabled,pct_disabled,block_grp_disabled_est
0,1388,6,47,1200,2,6,5012,428,0.085395,119
1,1531,6,47,1200,4,6,5012,428,0.085395,131
2,1405,6,47,1200,1,6,5012,428,0.085395,120
3,1312,6,47,1200,3,6,5012,428,0.085395,112
4,2410,6,47,1401,2,6,3727,295,0.079152,191
...,...,...,...,...,...,...,...,...,...,...
23206,2137,6,15,105,5,6,6093,641,0.105203,225
23207,480,6,15,105,4,6,6093,641,0.105203,50
23208,820,6,15,105,3,6,6093,641,0.105203,86
23209,1508,6,15,105,2,6,6093,641,0.105203,159


In [70]:
accessible_stops_trips.geometry = accessible_stops_trips.buffer(400)

In [71]:
map_hqta(accessible_stops_trips)

Map(center=[37.431428917115994, -122.09877658538603], controls=(ZoomControl(options=['position', 'zoom_in_text…